In [87]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import folium 
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors
import geocoder

In [149]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Remove any unassigned boroughs:

In [150]:
# Index into the dataframe where the Borough values are not unassigned.
df = (df[df["Borough"]!='Not assigned'])

# With the np.where(condition,x,y) function we can return an array of values from x when the condition
# is true and values from y where the condition is false.
# The following line replaces "Not assigned" values in Neighborhood with the corresponding values in Borough
df['Neighborhood'] = np.where(df['Neighborhood'] == "Not assigned", df['Borough'], df['Neighborhood'])

# This command groups the data frame based on the objects in "Postal Code" and then for all objects associated with each unique
# value of "Postal Code" we can aggregate them in some way. In this case the aggregation is defined by the following procedure:
# (1) use set(x) to get unique values, (2) cast the set to a list (3) using the "join" method convert the list into a string
# with elements seperated using ", " as a separator:
df = df.groupby(["Postal Code"]).agg(lambda x: ", ".join(list(set(x)))).reset_index()
df.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [151]:
print("The dimensions of the data frame are:", df.shape)

The dimensions of the data frame are: (103, 3)


The follow section adds the latitude and longitude coordinates for the corresponding postal codes (please note geocode is unreliable and may take an unpractically long time to run - see section below for alternative).

In [138]:
def get_geocode(postal_code):

    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    
    return latitude, longitude

In [139]:
la_list = []
lo_list = []

# loop through postal codes and append corresponding latitude and longitude coordinates to lists
for i in df["Postal Code"]:
    la, lo = get_geocode(i)
    la_list.append(la)
    lo_list.append(lo)

df["Latitude"] = la_list
df["Longitude"] = lo_list

Due to reliability issues with geocode we can load the coordinates using a .csv instead.

In [162]:
coord_df=pd.read_csv('http://cocl.us/Geospatial_data')
coord_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [163]:
df_coor = pd.merge(df,coord_df,on='Postal Code') # merge dataframes based on Postal Code
df_coor.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


We then select Borough in Toronto (thoses that contain the word Toronto)

In [303]:
tor_df=(df_coor[df_coor['Borough'].str.contains("Toronto")]).reset_index(drop = True)
tor_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


Define a function to get the near by venues for a given location

In [284]:
CLIENT_ID = 'FT1GRNDTS2PMIVIHVJJ1J3QUMRTQ5140BPQJAQ5S0AWE4AZK' # your Foursquare ID
CLIENT_SECRET = '420DMR2FMLHN2K4AIQ02BTWDRTOUO10INWPKZYWSOJFRLG4E' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

In [285]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

We pass the Neighborhood, Latitude and Longitude parameters from the data set as inputs into the getNearbyVenues function:

In [307]:
tor_venues = getNearbyVenues(names=tor_df['Neighborhood'],
                                   latitudes=tor_df['Latitude'],
                                   longitudes=tor_df['Longitude'])

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

The dataframe containing the venue assocaited with each neighborhood is:

In [308]:
tor_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


To run k-means clustering we convert the categorical data:

In [309]:
venue_onehot = pd.get_dummies(tor_venues[['Venue Category']], prefix="", prefix_sep="").reset_index(drop = True)
venue_onehot.head()

,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


We remove the neighborhood values from the onehot dataframe and inset the neighborhood values from the initial categorical dataframe:

In [310]:
venue_onehot.drop(['Neighborhood'],axis=1,inplace=True) 
venue_onehot.insert(loc=0, column='Neighborhood', value=tor_venues['Neighborhood'] )
venue_onehot.head()


,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


We can then group by neighborhood and using mean we get the % of venues for each venue category.

In [311]:
tor_grouped = venue_onehot.groupby('Neighborhood').mean().reset_index()
tor_grouped

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.066667,0.066667,0.066667,0.133333,0.133333,0.133333,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Get the most frequent venues categories for each neighborhood:

In [312]:
num_top_venues = 5

for hood in tor_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tor_grouped[tor_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0  Seafood Restaurant  0.07
1      Farmers Market  0.07
2        Cocktail Bar  0.07
3         Coffee Shop  0.07
4            Creperie  0.03


----Brockton, Parkdale Village, Exhibition Place----
                   venue  freq
0                   Café  0.12
1         Breakfast Spot  0.08
2                 Bakery  0.08
3  Performing Arts Venue  0.08
4            Coffee Shop  0.08


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
              venue  freq
0       Yoga Studio  0.06
1     Auto Workshop  0.06
2              Park  0.06
3  Recording Studio  0.06
4        Restaurant  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.13
1   Airport Service  0.13
2  Airport Terminal  0.13
3           Airport  0.07
4     Boat or Ferry  0.07


----Central Bay Street----
                ve

In [313]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [314]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_grouped['Neighborhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Farmers Market,Cocktail Bar,Seafood Restaurant,Fish Market,Tailor Shop,Beer Bar,Basketball Stadium,Museum,Bakery
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Bakery,Performing Arts Venue,Breakfast Spot,Nightclub,Stadium,Intersection,Bar,Burrito Place
2,"Business reply mail Processing Centre, South C...",Yoga Studio,Auto Workshop,Pizza Place,Comic Shop,Recording Studio,Restaurant,Burrito Place,Brewery,Light Rail Station,Skate Park
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Airport,Harbor / Marina,Coffee Shop,Plane,Rental Car Location,Sculpture Garden,Boat or Ferry
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Comic Shop,Spa,Bar,Bubble Tea Shop,Poke Place,Steakhouse


Perform the clustering based on the frequency of venue categories in each neighborhood:

In [315]:
# set number of clusters
kclusters = 10

tor_grouped_clustering = tor_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=10).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 6, 0, 5, 0, 6, 5, 6, 9])

In [316]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tor_merged = tor_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

tor_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,7,Trail,Pub,Health Food Store,Yoga Studio,Creperie,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,6,Greek Restaurant,Restaurant,Italian Restaurant,Ice Cream Shop,Yoga Studio,Cosmetics Shop,Bookstore,Juice Bar,Dessert Shop,Spa
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,6,Park,Pizza Place,Pet Store,Fast Food Restaurant,Movie Theater,Pub,Liquor Store,Restaurant,Burrito Place,Sandwich Place
3,M4M,East Toronto,Studio District,43.659526,-79.340923,5,Café,Coffee Shop,Bakery,Bookstore,Seafood Restaurant,Sandwich Place,Brewery,Cheese Shop,Park,Yoga Studio
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Park,Bus Line,Swim School,Yoga Studio,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store


Get the Toronto coordinates for map visulisation:

In [317]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [318]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighborhood'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [319]:
tor_merged.loc[tor_merged['Cluster Labels'] == 0, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Central Toronto,0,Clothing Store,Coffee Shop,Restaurant,Sporting Goods Shop,Spa,Shoe Store,Diner,Seafood Restaurant,Salon / Barbershop,Rental Car Location
14,Downtown Toronto,0,Café,Clothing Store,Theater,Tea Room,Gastropub,Sandwich Place,Diner,Coffee Shop,Shopping Mall,Burger Joint
16,Downtown Toronto,0,Coffee Shop,Farmers Market,Cocktail Bar,Seafood Restaurant,Fish Market,Tailor Shop,Beer Bar,Basketball Stadium,Museum,Bakery
19,Downtown Toronto,0,Café,Plaza,Park,Hotel,Ice Cream Shop,Dance Studio,Sporting Goods Shop,Basketball Stadium,IT Services,Performing Arts Venue
25,Downtown Toronto,0,Café,Restaurant,Sandwich Place,Bookstore,Japanese Restaurant,Bar,Bakery,Yoga Studio,Sushi Restaurant,College Arts Building
26,Downtown Toronto,0,Café,Mexican Restaurant,Vietnamese Restaurant,Bakery,Comfort Food Restaurant,Belgian Restaurant,Record Shop,Caribbean Restaurant,Cheese Shop,Coffee Shop
27,Downtown Toronto,0,Airport Lounge,Airport Service,Airport Terminal,Airport,Harbor / Marina,Coffee Shop,Plane,Rental Car Location,Sculpture Garden,Boat or Ferry
28,Downtown Toronto,0,Café,Restaurant,Cocktail Bar,Beer Bar,Farmers Market,Seafood Restaurant,Tailor Shop,Pub,Museum,Hotel
30,Downtown Toronto,0,Grocery Store,Café,Park,Baby Store,Nightclub,Candy Store,Restaurant,Diner,Italian Restaurant,Coffee Shop
31,West Toronto,0,Pharmacy,Bakery,Supermarket,Pet Store,Music Venue,Middle Eastern Restaurant,Café,Brewery,Brazilian Restaurant,Bar


In [320]:
tor_merged.loc[tor_merged['Cluster Labels'] == 1, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,1,Pool,Garden,Home Service,Yoga Studio,Coworking Space,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store


In [321]:
tor_merged.loc[tor_merged['Cluster Labels'] == 2, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,2,Park,Bus Line,Swim School,Yoga Studio,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store


In [322]:
tor_merged.loc[tor_merged['Cluster Labels'] == 3, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,3,Jewelry Store,Park,Trail,Sushi Restaurant,Yoga Studio,Creperie,Distribution Center,Discount Store,Diner,Dessert Shop


In [323]:
tor_merged.loc[tor_merged['Cluster Labels'] == 4, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,4,Playground,Restaurant,Yoga Studio,Coworking Space,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega


In [324]:
tor_merged.loc[tor_merged['Cluster Labels'] == 5, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,East Toronto,5,Café,Coffee Shop,Bakery,Bookstore,Seafood Restaurant,Sandwich Place,Brewery,Cheese Shop,Park,Yoga Studio
11,Downtown Toronto,5,Italian Restaurant,Bakery,Coffee Shop,Restaurant,Café,Deli / Bodega,Liquor Store,Pub,Caribbean Restaurant,Bank
13,Downtown Toronto,5,Coffee Shop,Park,Bakery,Café,Breakfast Spot,Yoga Studio,Historic Site,Distribution Center,Restaurant,Spa
15,Downtown Toronto,5,Gastropub,Restaurant,Japanese Restaurant,Café,Coffee Shop,Gym,Italian Restaurant,Ice Cream Shop,Beer Bar,Hotel
17,Downtown Toronto,5,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Comic Shop,Spa,Bar,Bubble Tea Shop,Poke Place,Steakhouse
18,Downtown Toronto,5,Coffee Shop,Café,Hotel,Pizza Place,Asian Restaurant,Salon / Barbershop,Seafood Restaurant,Juice Bar,Monument / Landmark,Smoke Shop
20,Downtown Toronto,5,Coffee Shop,Café,Japanese Restaurant,Restaurant,Sandwich Place,Gym,Pub,Bakery,Beer Bar,Steakhouse
21,Downtown Toronto,5,Café,Gastropub,Japanese Restaurant,Coffee Shop,Hotel,Bakery,Sandwich Place,Seafood Restaurant,Bookstore,Museum
24,Central Toronto,5,Sandwich Place,Café,Coffee Shop,Middle Eastern Restaurant,Donut Shop,Burger Joint,Liquor Store,Indian Restaurant,BBQ Joint,Shoe Repair
29,Downtown Toronto,5,Café,Coffee Shop,Restaurant,Seafood Restaurant,Hotel,Tea Room,Gastropub,Gym,Bookstore,Pub


In [325]:
tor_merged.loc[tor_merged['Cluster Labels'] == 6, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,6,Greek Restaurant,Restaurant,Italian Restaurant,Ice Cream Shop,Yoga Studio,Cosmetics Shop,Bookstore,Juice Bar,Dessert Shop,Spa
2,East Toronto,6,Park,Pizza Place,Pet Store,Fast Food Restaurant,Movie Theater,Pub,Liquor Store,Restaurant,Burrito Place,Sandwich Place
7,Central Toronto,6,Sandwich Place,Dessert Shop,Coffee Shop,Sushi Restaurant,Pizza Place,Café,Gym,Italian Restaurant,Park,Gas Station
9,Central Toronto,6,Coffee Shop,Pub,Supermarket,Burger Joint,Restaurant,Light Rail Station,Liquor Store,Bank,Sports Bar,Bagel Shop
12,Downtown Toronto,6,Coffee Shop,Restaurant,Sushi Restaurant,Pub,Ice Cream Shop,Smoke Shop,Café,Beer Bar,Dance Studio,Bookstore
35,West Toronto,6,Gift Shop,Breakfast Spot,Coffee Shop,Cuban Restaurant,Bookstore,Dog Run,Bar,Italian Restaurant,Dessert Shop,Restaurant
36,West Toronto,6,Café,Italian Restaurant,Coffee Shop,Pizza Place,Pub,Sushi Restaurant,Sandwich Place,Bar,Smoothie Shop,Indie Movie Theater
38,East Toronto,6,Yoga Studio,Auto Workshop,Pizza Place,Comic Shop,Recording Studio,Restaurant,Burrito Place,Brewery,Light Rail Station,Skate Park


In [326]:
tor_merged.loc[tor_merged['Cluster Labels'] == 7, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,7,Trail,Pub,Health Food Store,Yoga Studio,Creperie,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop


In [327]:
tor_merged.loc[tor_merged['Cluster Labels'] == 8, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Downtown Toronto,8,Park,Playground,Trail,Yoga Studio,Coworking Space,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store


In [328]:
tor_merged.loc[tor_merged['Cluster Labels'] == 9, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Central Toronto,9,Hotel,Pizza Place,Park,Breakfast Spot,Gym,Department Store,Sandwich Place,Food & Drink Shop,Discount Store,Diner
